IMPORTS

In [120]:
import pandas as pd
import datetime as dt
import numpy as np
import ast

BASES

MERGES et BASES

In [121]:
df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/343300649.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t')


In [122]:
df_title_basics = df_title_basics.replace('\\N', np.nan)

In [123]:
df_title_ratings = pd.read_csv('BD/title.ratings.tsv.gz', compression = 'gzip', sep = '\t')

In [124]:
df_title_ratings = df_title_ratings.replace('\\N', np.nan)

In [125]:
df1 = pd.merge(df_title_basics,
               df_title_ratings,
               how = 'outer',
               on = 'tconst')

In [126]:
df_title_ratings, df_title_basics = 0, 0

In [127]:
df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t')

In [128]:
df_title_akas = df_title_akas.replace('\\N', np.nan)

In [129]:
df_3 = pd.merge(df1,
                df_title_akas,
                how = 'outer',
                left_on = 'tconst',
                right_on = 'titleId')

In [130]:
df_3 = df_3[(df_3['titleType'] == 'movie')]

In [131]:
df1, df_title_akas = 0, 0

In [132]:
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/1088705580.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv')


In [133]:
df_tmbd_full = df_tmbd_full.replace('\\N', np.nan)

In [134]:
df_films = pd.merge(df_tmbd_full,
                df_3,
                how = 'outer',
                left_on = 'imdb_id',
                right_on = 'tconst')

In [135]:
df_tmbd_full, df_3 = 0, 0

COLONNES NUMERIQUES

In [136]:
# On remplit les cellules vides par 0

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/2166249025.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].fillna(0)


In [137]:
# On remplace les '\\N' par 0

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].replace('\\N', 0)

In [138]:
# On met ces colonnes au format numérique

df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']] = df_films[['runtimeMinutes', 'budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count', 'startYear', 'endYear', 'averageRating', 'numVotes']].apply(pd.to_numeric)

In [139]:
# On transforme les types de données pour 'release_date'

df_films['release_date'] = pd.to_datetime(df_films['release_date'])
df_films['release_date'] = df_films['release_date'].replace('\\N', 0)
df_films['release_date'] = df_films['release_date'].dt.year

In [140]:
# On transforme les types de données pour 'startYear'

df_films['startYear'] = df_films['startYear'].replace('\\N', 0)
df_films['startYear'] = pd.to_numeric(df_films['startYear'], downcast="integer")

In [141]:
# On crée la colonne year_exact

df_films['year_exact'] = df_films[['release_date', 'startYear']].max(axis = 1)

In [142]:
# On crée la colonne 'Decennie'

df_films['Decennie'] = ''
df_films['year_exact'] = df_films['year_exact'].fillna('0')
df_films['year_exact'] = df_films['year_exact'].replace('nan', '0')
df_films['year_exact'] = df_films['year_exact'].astype(str)

In [143]:
df_films['Decennie'] = df_films['year_exact'].str[:3] + "0"
# df_films['Decennie'] = df_films['Decennie'].replace("0.00's", None)
df_films['year_exact'] = pd.to_numeric(df_films['year_exact'])
df_films['Decennie'] = pd.to_numeric(df_films['Decennie'])

RUNTIME

In [144]:
# On met de côté les valeurs nulles

df_null1 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_null2 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'] == 0))]
df_null3 = df_films[((df_films['runtimeMinutes'] == 0) & (df_films['runtime'].isna()))]
df_null4 = df_films[((df_films['runtimeMinutes'].isna()) & (df_films['runtime'] == 0))]

In [145]:
df_null = pd.concat([df_null1, df_null2, df_null3, df_null4])

In [146]:
df_null1, df_null2, df_null3, df_null4 = 0, 0, 0, 0

In [147]:
# On prend les valeurs utiles

df_utile = df_films[~((df_films['runtimeMinutes'].isna()) & (df_films['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'] == 0))]
df_utile = df_utile[~((df_utile['runtimeMinutes'] == 0) & (df_utile['runtime'].isna()))]
df_utile = df_utile[~((df_utile['runtimeMinutes'].isna()) & (df_utile['runtime'] == 0))]

In [148]:
# Calcul différence de runtime des 2 tables

df_utile['difference_runtime'] = abs(df_utile['runtime'] - df_utile['runtimeMinutes'])

In [149]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_utile[(df_utile['runtimeMinutes'] == df_utile['runtime'])]

In [150]:
# Création colonne RuntimeExact
df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']

df_a_travailler_1 = df_utile[~(df_utile['runtimeMinutes'] == df_utile['runtime'])]

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/3570380868.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['runtime_exact'] = df_a_garder_1['runtimeMinutes']


In [151]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]

In [152]:
# On conserve la donnée existante

df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/3521993277.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['runtime_exact'] = df_a_garder_2[['runtimeMinutes', 'runtime']].max(axis = 1)


In [153]:
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['runtimeMinutes'].isna()) | (df_a_travailler_1['runtime'].isna())| (df_a_travailler_1['runtime'] == 0) | (df_a_travailler_1['runtimeMinutes'] == 0))]
df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/1261991755.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_runtime'] = abs(df_a_travailler_2['runtime'] - df_a_travailler_2['runtimeMinutes'])


In [154]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_runtime'] <= 20]

In [155]:
# On prend comme base la valeur max des 2 colonnes

df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/880983650.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_3['runtime_exact'] = df_a_garder_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [156]:
df_a_travailler_3 = df_a_travailler_2[~(df_a_travailler_2['difference_runtime'] <= 20)]

In [157]:
# Pour le restant des données, on conserve la valeur max des 2 colonnes

df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/264539281.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['runtime_exact'] = df_a_travailler_3[['runtimeMinutes', 'runtime']].max(axis = 1)


In [158]:
df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])

In [159]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [160]:
df_films = pd.concat([df_final, df_null])

In [161]:
df_final, df_null = 0, 0

VOTES

In [162]:
# On met de côté les valeurs inutiles

df_null5 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_null6 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'] == 0))]
df_null7 = df_films[((df_films['vote_average'] == 0) & (df_films['averageRating'].isna()))]
df_null8 = df_films[((df_films['vote_average'].isna()) & (df_films['averageRating'] == 0))]

In [163]:
df_null9 = pd.concat([df_null5, df_null6, df_null7, df_null8])

In [164]:
df_null5, df_null6, df_null7, df_null8 = 0, 0, 0, 0

In [165]:
# On prend les valeurs utiles

df_ratings = df_films[~((df_films['vote_average'].isna()) & (df_films['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'] == 0))]
df_ratings = df_ratings[~((df_ratings['vote_average'] == 0) & (df_ratings['averageRating'].isna()))]
df_ratings = df_ratings[~((df_ratings['vote_average'].isna()) & (df_ratings['averageRating'] == 0))]

In [166]:
df_ratings['vote_average'] = round(df_ratings['vote_average'], 1)

In [167]:
# On met de côté là où les 2 valeurs sont identiques

df_a_garder_1 = df_ratings[df_ratings['averageRating'] == df_ratings['vote_average']]
df_a_travailler_1 = df_ratings[~(df_ratings['averageRating'] == df_ratings['vote_average'])]

In [168]:
# On met de côté là où une valeur manque seulement

df_a_garder_2 = df_a_travailler_1[(df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0)]
df_a_travailler_2 = df_a_travailler_1[~((df_a_travailler_1['averageRating'].isna()) | (df_a_travailler_1['vote_average'].isna())| (df_a_travailler_1['vote_average'] == 0) | (df_a_travailler_1['averageRating'] == 0))]

In [169]:
df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/2546417221.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_2['difference_vote'] = abs(df_a_travailler_2['averageRating'] - df_a_travailler_2['vote_average'])


In [170]:
# On met de côté là où la différence de durée est négligeable

df_a_garder_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] <= 1]
df_a_travailler_3 = df_a_travailler_2[df_a_travailler_2['difference_vote'] > 1]

In [171]:
df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
df_a_travailler_3['vote_exact'] = round(df_a_travailler_3['vote_exact'], 1)
df_a_garder_4 = df_a_travailler_3

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/4062215657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travailler_3['vote_exact'] = ((df_a_travailler_3['vote_count'] * df_a_travailler_3['vote_average']) + (df_a_travailler_3['numVotes'] * df_a_travailler_3['averageRating'])) / (df_a_travailler_3['numVotes'] + df_a_travailler_3['vote_count'])
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/4062215657.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_travai

In [172]:
# On crée la colonne vote_exact pour garder la moyenne des votes

df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
df_a_garder_3['vote_exact'] = df_a_garder_3[['averageRating', 'vote_average']].mean(axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/3653439724.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_1['vote_exact'] = df_a_garder_1[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/3653439724.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_a_garder_2['vote_exact'] = df_a_garder_2[['averageRating', 'vote_average']].max(axis = 1)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/36534

In [173]:
# On concatène tous les tableaux conservés

df_final = pd.concat([df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4])
df_final['arrondi_vote_exact'] = round(df_final['vote_exact'], 0)

In [174]:
df_a_travailler_1, df_a_travailler_2, df_a_travailler_3 = 0, 0, 0
df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4 = 0, 0, 0, 0

In [175]:
df_films = pd.concat([df_final, df_null9])

In [176]:
df_final, df_null9 = 0, 0

GENRE

In [177]:
df_films = df_films.rename(columns = {'genres_x' : 'genres_y', 'genres_y' : 'genres_x'})

In [178]:
df_films[df_films[['genres_y', 'genres_x']] == '\\N'] = df_films[df_films[['genres_y', 'genres_x']] == '\\N'].replace('\\N', np.nan)

In [179]:
def transfo_liste(x):
  if isinstance(x, str):
    return ast.literal_eval(x)
  else:
    return x
  
df_films['genres_y'] = df_films['genres_y'].apply(transfo_liste)

In [180]:
df_films['genres_y'] = df_films['genres_y'].apply(lambda x: x if (type(x) == list) else [])

In [181]:
df_films['genres_x'] = df_films['genres_x'].str.replace(",", ", ")

In [182]:
df_films['genres_x_list'] = df_films['genres_x'].fillna('').str.split(', ')

In [183]:
df_films['genre'] = df_films.apply(lambda row: list(set(row['genres_y'] + row['genres_x_list'])),axis=1)

In [ ]:
df_films['genre'] = df_films['genre'].apply(lambda x: x if x != [] else np.nan)

In [184]:
df_films = df_films.drop(['genres_x_list', 'genres_x', 'genres_y'], axis = 1)

TITLE

In [185]:
# df_films['title'] = df_films.apply(lambda row : row['title_y'] if (row['region'].str.contains('FR') == True) else row['title_y'].fillna(row['title_x']).fillna(row['primaryTitle']).fillna(row['original_title']).fillna(row['originalTitle']))

df_films['title'] = df_films['title_y'].fillna(df_films['title_x']).fillna(df_films['primaryTitle']).fillna(df_films['original_title']).fillna(df_films['originalTitle'])
df_films = df_films.drop(['original_title', 'title_y', 'title_x', 'primaryTitle', 'originalTitle'], axis = 1)

ADULT

In [186]:
df_films['adult'] = (df_films['isAdult'] == 1) | df_films['adult']
df_films = df_films.drop(['isAdult'], axis=1)

TCONST

In [187]:
df_films['film_id'] = df_films['imdb_id'].fillna(df_films['tconst']).fillna(df_films['titleId'])

In [188]:
df_films = df_films.drop(['imdb_id', 'tconst', 'titleId'], axis = 1)

VOTE COUNT

In [189]:
df_films['vote_count_mean'] = (df_films['vote_count'] + df_films['numVotes'])/2

In [190]:
df_films = df_films.drop(['runtime', 'release_date', 'vote_average', 'numVotes', 'vote_count', 'startYear', 'endYear', 'runtimeMinutes', 'averageRating', 'difference_runtime', 'difference_vote'], axis = 1)

FINAL LANGUAGE

In [191]:
colonnes_y = ['production_countries', 'spoken_languages', 'production_companies_country']

for cl in colonnes_y:
 
    df_films[cl] = df_films[cl].apply(transfo_liste)
    df_films[cl] = df_films[cl].apply(lambda x: x if (type(x) == list) else [])

In [192]:
df_films['original_language_x_list'] = df_films['original_language'].fillna('').str.split(', ')

In [193]:
df_films['language_x_list'] = df_films['language'].fillna('').str.split(', ')

In [194]:
df_films['final_language'] = df_films.apply(lambda row: list(set(row['production_countries'] + row['spoken_languages'] + row['production_companies_country'] + row['original_language_x_list'] + row['language_x_list'])),axis=1)

In [195]:
df_films['final_language'] = df_films['final_language'].apply(lambda x: x if x != [] else np.nan)

In [196]:
df_films['final_language'] = df_films['final_language'].apply(lambda x: ', '.join(x) if type(x) == list else x)

In [197]:
df_films = df_films.drop(['original_language', 'spoken_languages', 'language', 'original_language_x_list', 'language_x_list'], axis = 1)

In [198]:
df_films = df_films[(df_films['final_language'].isna() == True) | (df_films['final_language'] == '') | (df_films['final_language'].str.contains('fr') == True) | (df_films['final_language'].str.contains('FR') == True)  | (df_films['final_language'].str.contains('us') == True) | (df_films['final_language'].str.contains('US') == True)]

CLEAN FINAL DU MERGE ET GROUP BY

In [199]:
df_str = df_films.select_dtypes(exclude = 'number')

In [200]:
df_str['language_fr'] = df_str['final_language'].apply(lambda x : 1 if ('fr' in x or 'FR' in x) else 0)

In [201]:
df_str = df_str.sort_values(by = 'language_fr', ascending = False)

In [202]:
df_str_drop_dup = df_str.drop_duplicates(subset = ['film_id'], keep = 'first')

In [203]:
df_num = pd.concat([df_films['film_id'], df_films.select_dtypes(include = 'number')], axis = 1)
df_num = df_num.drop(['revenue', 'budget'], axis = 1)

In [204]:
df_revenue_budget = df_films[['film_id', 'revenue', 'budget']]
df_revenue_budget_gb = df_revenue_budget.groupby('film_id').max()

In [205]:
df_num_gb = df_num.groupby('film_id').max()

In [206]:
df_num2 = pd.merge(df_num_gb,
                   df_revenue_budget_gb,
                   how = 'outer',
                   on = 'film_id')

In [207]:
df_films = pd.merge(df_str_drop_dup,
                   df_num2,
                   on = 'film_id',
                   how = 'left')

In [208]:
df_num2, df_str_drop_dup, df_num_gb, df_revenue_budget_gb, df_revenue_budget, df_num, df_str = 0, 0, 0, 0, 0, 0, 0

In [209]:
df_films = df_films[df_films['status'] == 'Released']

In [210]:
df_films = df_films.drop(['isOriginalTitle', 'revenue', 'budget', 'ordering', 'types', 'attributes', 'video', 'id'], axis = 1)

In [211]:
df_films = df_films[~(df_films['vote_count_mean'] == 0)] # A voir si on le conserve
df_films = df_films[~(df_films['year_exact'] == 0)]
df_films = df_films[~(df_films['runtime_exact'].isna() == True)]
df_films['region'] = df_films['region'].fillna('Unknown')

In [212]:
df_films = df_films.rename(columns = {
    'backdrop_path' : 'backdrop_path_out_KNN',
    'homepage' : 'homepage_out_KNN',
    'overview' : 'overview_out_KNN',
    'poster_path' : 'poster_path_out_KNN',
    'film_id' : 'film_id_out_KNN',
    'title' : 'title_out_KNN',
})

CHANGEMENT GENRE

In [213]:
tous_les_genres = set()
for genres in df_films['genre']:
  tous_les_genres.update(genres)

In [214]:
tous_les_genres.remove("")

In [215]:
for genre in tous_les_genres:
  df_films[f'genre_{genre}'] = df_films['genre'].apply(lambda x: genre in x)

In [216]:
df_films = df_films.drop('genre', axis = 1)

CHANGEMENT production_companies_name

In [217]:
# df_films['production_companies_name2'] = df_films['production_companies_name'].apply(lambda x: x.replace("[", '').replace("]", '').replace("'", ''))

# df_films['production_companies_name3'] = df_films['production_companies_name2'].apply(lambda x : x.split(','))

# tous_les_genres = set()
# for genres in df_films['production_companies_name3']:
#   tous_les_genres.update(genres)

# tous_les_genres.remove("")

# for genre in tous_les_genres:
#   df_films[f'production_companies_name_{genre}'] = df_films['production_companies_name3'].apply(lambda x: genre in x)

# df_films = df_films.drop(['production_companies_name', 'production_companies_name2', 'production_companies_name3'], axis = 1)

CHANGEMENT production_countries

In [218]:
# df_films['production_countries2'] = df_films['production_countries'].apply(lambda x: x.replace("['", '').replace("']", '').replace("'", ''))
# df_films['production_countries3'] = df_films['production_countries2'].apply(lambda x : x.split(','))


In [219]:
tous_les_genres = set()
for genres in df_films['production_countries']:
  tous_les_genres.update(genres)

In [220]:
for genre in tous_les_genres:
  df_films[f'production_countries_{genre}'] = df_films['production_countries'].apply(lambda x: genre in x)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/308899010.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'production_countries_{genre}'] = df_films['production_countries'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/308899010.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'production_countries_{genre}'] = df_films['production_countries'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/30889

In [221]:
df_films = df_films.drop(['production_countries'], axis = 1)

CHANGEMENT production_companies_country

In [222]:
# df_films['production_companies_country2'] = df_films['production_companies_country'].apply(lambda x: x.replace("['", '').replace("']", '').replace("'", ''))
# df_films['production_companies_country3'] = df_films['production_companies_country2'].apply(lambda x : x.split(','))

In [223]:
df_films['production_companies_country2'] = df_films['production_companies_country'].apply(lambda x : ",".join(x).lower())

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/3094971156.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films['production_companies_country2'] = df_films['production_companies_country'].apply(lambda x : ",".join(x).lower())


In [224]:
df_films['production_companies_country3'] = df_films['production_companies_country2'].apply(lambda x : x.split(','))

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/1064447305.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films['production_companies_country3'] = df_films['production_companies_country2'].apply(lambda x : x.split(','))


In [225]:
tous_les_genres = set()
for genres in df_films['production_companies_country3']:
  tous_les_genres.update(genres)

In [226]:
tous_les_genres.remove('')

In [227]:
for genre in tous_les_genres:
  df_films[f'production_companies_country_{genre}'] = df_films['production_companies_country'].apply(lambda x: genre in x)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/2202528436.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'production_companies_country_{genre}'] = df_films['production_companies_country'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/2202528436.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'production_companies_country_{genre}'] = df_films['production_companies_country'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vj

In [228]:
df_films = df_films.drop(['production_companies_country', 'production_companies_country3', 'production_companies_country2'], axis = 1)

In [229]:
df_films = df_films.rename(columns = {'final_language' : 'final_language_out_KNN', 'tagline' : 'tagline_out_KNN', 'production_companies_country FR': 'production_companies_countryfr'})

CHANGEMENT POUR LANGUE FR

In [230]:
df_films['prod_FR'] = df_films.apply(lambda row : 1 if row['production_companies_country_fr'] == True else 0, axis = 1)

In [231]:
# df_films['prod_FR'] = df_films.apply(lambda row : 1 if (('fr' in row['production_companies_country'] == True) or ('FR' in row['production_companies_country'])) else 0, axis = 1)
# df_films = df_films.drop(['production_companies_countryfr', 'production_companies_countryFR'], axis = 1)

CHANGEMENT POUR FILM US

In [232]:
df_films['prod_US'] = df_films.apply(lambda row : 1 if row['production_companies_country_us'] == True else 0, axis = 1)

In [233]:
# df_films['prod_US'] = df_films.apply(lambda row : 1 if (row['production_companies_countryUS'] == True) or (row['production_companies_country US'] == True) else 0, axis = 1)
# df_films = df_films.drop(['production_companies_countryUS', 'production_companies_country US'], axis = 1)

In [234]:
df_films = df_films.drop('production_companies_country_us', axis = 1)

In [235]:
df1, df_3, df_a_garder_1, df_a_garder_2, df_a_garder_3, df_a_garder_4, df_a_travailler_1, df_a_travailler_2, df_a_travailler_3, df_final, df_null = 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [236]:
df_films[df_films.select_dtypes('number') == True] = df_films[df_films.select_dtypes('number') == True].fillna(0)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_16607/3129310626.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_films[df_films.select_dtypes('number') == True] = df_films[df_films.select_dtypes('number') == True].fillna(0)


In [237]:
df_films[['vote_exact', 'arrondi_vote_exact']] = df_films[['vote_exact', 'arrondi_vote_exact']].astype(float)

In [238]:
df_films['titleType'] = df_films['titleType'].fillna('movie')

EXPORT

In [239]:
df_films.to_csv('P2_G5_films.csv.gz', index=False, compression='gzip')